# Age predictor
The below program uses CNN to predict whether the person in photo is 
**YOUNG** , **MIDDLE** or **OLD**  
*(numerical age predictions might be not very accurate on the data set)*

## dependencies 
in this program we have used libraries commonly used in python machine learning problems 

- Pandas
    * pandas is an open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.
- numpy
    * NumPy is the fundamental package for scientific computing with Python.
- openCV
    * OpenCV is a library to handle realtime applications of computer vision as well as image processing
- matplotlib
    * Matplotlib is a plotting library for the Python programming language 
- tensorflow
    * Open source machine learning library for research as well as production


In [1]:
import cv2 as cv2

import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import datetime , os

## getting model  training data
having an appropriate dataset is key to any machine learning problem
This dataset was obtained from [Analytics Vidhya](https://www.analyticsvidhya.com/).
We have a .csv file that contains the *ID* of images of various faces and along with which *Class* do they fall into.

In [2]:
data = pd.read_csv("train.csv")
data.head()


,ID,Class
0,377.jpg,MIDDLE
1,17814.jpg,YOUNG
2,21283.jpg,MIDDLE
3,16496.jpg,YOUNG
4,4487.jpg,MIDDLE


In [3]:
data.shape

(19906, 2)

### Encoding class into a numerical form
This is a classification problem.
We create a column **age** and put values in age based on class to encode the classes. 

- if **Class** == 'YOUNG' , **age** = 2
- if **Class** == 'MIDDLE' , **age** = 0
- if **Class** == 'OLD' , **age** = 1

In [4]:
data['Class']=data['Class'].astype('category')
data['age'] = data['Class'].cat.codes

In [5]:
data.head()

,ID,Class,age
0,377.jpg,MIDDLE,0
1,17814.jpg,YOUNG,2
2,21283.jpg,MIDDLE,0
3,16496.jpg,YOUNG,2
4,4487.jpg,MIDDLE,0


Function to show a image 

In [6]:
def show_photo(img):
    plt.imshow(img,cmap='gray')
    plt.show()

## Preparing the training set
Now we use ID from our csv data to load the set of images that we will use for training the model
We will load the pictures in grayscale readjust the photos so that they are the same size.
We will normalize the the picture data to ease with our computations

In [7]:
photos = [cv2.resize(cv2.imread("Train/"+s ,  0),(40,40)) for s in data["ID"]]
photos = [tf.keras.utils.normalize(p) for p in photos]

photos = np.array(photos)
photos = photos.reshape(19906,40,40,1)
photos.shape

(19906, 40, 40, 1)

## Adding the layers to model
We have used 4 convolution layers and 3 dense layers , two with rectified linear unit activation and the last with a softmax activation as its the output layer of a classification problem

In [8]:
from tensorflow.keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Dropout

model = tf.keras.models.Sequential()
model.add(Conv2D(32, kernel_size = (3,3) , activation = 'relu' , input_shape = (40,40,1)))
model.add(Conv2D(40, kernel_size = (3,3) , activation = 'relu' ))
model.add(Conv2D(64, kernel_size = (3,3) , activation = 'relu'  ))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(80, kernel_size = (3,3) , activation = 'relu'  ))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation ='relu'))
model.add(Dropout(0.25))
model.add(Dense(64, activation ='relu'))
model.add(Dropout(0.4))
model.add(Dense(3,activation = "softmax"))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Here we have used a Adam optimizer , loss is calculated by sparse categorical cross entrophy

In [22]:

model.compile(optimizer = "adam" , loss = "sparse_categorical_crossentropy" , metrics = ["accuracy"])


In [10]:
xtrain=photos
ytrain = data["age"]


In [28]:
model.fit(xtrain,ytrain,epochs = 1, batch_size = 128)


19906/19906 [==============================] - 74s 4ms/sample - loss: 0.0715 - acc: 0.9743


## Testing
we use the test data to test our predictions
For that we will adjust the test data like we did the training data
and store the results for all the test data

In [18]:

data_test = pd.read_csv('test.csv')
test = [cv2.resize(cv2.imread("Test/"+s, 0),(40,40)) for s in data_test['ID']]
test = [tf.keras.utils.normalize(p) for p in test]
test = np.array(test)
test = test.reshape((6636,40,40,1))
test.shape

predictions = model.predict_classes(test)

images = [cv2.imread("Test/"+s, 0) for s in data_test['ID']]


age = list(predictions)

for i in range(len(age)):
    if age[i] == 0:
        age[i] = 'MIDDLE'
    elif age[i] == 1:
        age[i] = 'OLD'
    else:
        age[i] = 'YOUNG'

show_photo(images[420])
age[420]

## Saving the model
we save the model in a json file and also save the model weights

In [27]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")

### Extra : Loading externally saved model

In [13]:
from tensorflow.keras.models import model_from_json

json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("model.h5")